## Numerical Representations of Words
The Word2Vec model trained in Part2 consists of a feature vector for each word in the vocabulary, stored in a numpy array called 'syn0'.

Ref: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-3-more-fun-with-word-vectors

In [2]:
#Load the model that we created in Part2
from gensim.models import Word2Vec
model = Word2Vec.load('300features_40minwords_10context')


In [3]:
type(model.syn0)

numpy.ndarray

In [4]:
model.syn0.shape

(16731, 300)

Setting the minimum word count to 40 gave us a total vocabulary of 16731 words with 300 feature apiece.

In [5]:
# access individual word vectors
model['flower']

array([  7.03130066e-02,   5.14342189e-02,  -3.80656198e-02,
         4.53658774e-02,  -2.73839328e-02,  -5.64631633e-02,
         3.78377177e-02,   5.36779314e-02,   4.48896326e-02,
        -4.83323680e-03,   5.59773184e-02,  -8.08055792e-03,
         1.10543892e-01,  -1.57461375e-01,   1.12796193e-02,
        -9.08770263e-02,  -1.11438870e-01,   2.28472278e-02,
         4.86316085e-02,  -3.51550207e-02,  -4.96995123e-03,
        -5.61099909e-02,   5.66338301e-02,   5.45456521e-02,
         4.19501700e-02,   9.22863465e-03,  -2.02877950e-02,
         3.95046510e-02,  -7.24636689e-02,   1.89160760e-02,
        -8.77127126e-02,   4.20929427e-04,   4.82679084e-02,
        -1.74988247e-02,  -1.67242736e-02,  -9.87018719e-02,
         1.22327253e-01,  -3.90022546e-02,   1.57671347e-02,
        -3.52325179e-02,  -3.57532203e-02,  -6.47378862e-02,
        -6.03422076e-02,   7.79911061e-04,  -5.47066629e-02,
        -4.79701161e-02,   6.14088178e-02,   6.47710450e-03,
         4.84411418e-02,

## From Words to Paragraohs, Attempt1: Vector Averaging
Fisrt, we need to take individual word vectors and transform them into a feature set that is the same length for every review. Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method is to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

In [6]:
import numpy as np 
def makeFeatureVec(words,model,num_features):
    #Function to average all of the word vectors in a given paragraph
    #Pre-initialize an empty numpy array (for speed)
    featureVec= np.zeros((num_features,),dtype='float32')
    nwords=0
    #Index2word is a list that contains the names of the words in the model's vocabulary. Convert it to a set, for speed
    index2word_set=set(model.index2word)
    #Loop over each word in the review and, if it is in the model's vocabulary, and its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords+=1
            featureVec=np.add(featureVec,model[word])
    #Divide the result by the number of words to get the average
    featureVec =np.divide(featureVec,nwords)
    return featureVec
def getAvgFeatureVecs(reviews,model,num_features):
    #Given a set of reviews (each one a list of words), calculate the average feature vector for each one and return a 2D numpy array
    #Initialize a counter
    counter =0
    #Preallocate a 2D numpy array, for speed
    reviewFeatureVecs =np.zeros((len(reviews),num_features),dtype='float32')
    #Loop through the reviews
    for review in reviews:
        #print a status message every 1000th review
        if counter%1000.==0.:
            print ("Review %d of %d" %(counter, len(reviews)))
        #Call the function (defined above), that makes avearge feature vectors 
        reviewFeatureVecs[counter]=makeFeatureVec(review,model,num_features)
        #Increment the counter
        counter+=1
    return reviewFeatureVecs

**Recall Bag_of_words_p2**

In [7]:
import pandas as pd

# Read data from files
train=pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t",quoting=3)
test=pd.read_csv("testData.tsv", header=0, delimiter="\t",quoting=3)
unlabeled_train=pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)

# Import various modules for string cleaning 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.
    # 1. Remove HTML
    review_text=BeautifulSoup(review,"html").get_text()
    # 2. Remove non-letters, keep numbers
    review_text=re.sub("[^a-zA-Z0-9]"," ",review_text)
    # 3. Convert words in lower case and split them
    words=review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops=set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
    # 5. return a list of words
    return(words)

In [8]:
# Calculate average feature vectors for training and testing sets. Notice stop word is removed
num_features=300

clean_train_reviews=[]
for review in train['review']:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True)
    )
trainDataVecs=getAvgFeatureVecs(clean_train_reviews,model,num_features)

print('Creating average feature vecs for test reviews')
clean_test_reviews=[]
for review in test['review']:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True)
    )
testDataVecs=getAvgFeatureVecs(clean_test_reviews,model,num_features)

/home/amywu/anaconda3/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

Next, use the average paragraph vectors to train a random forest. Note that, as in part1, we can only use the labeled training reviews to train the model.

In [9]:
#Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100)

print('Fitting a random forest to labeled traning data...')
forest=forest.fit(trainDataVecs,train['sentiment'])

#Test and extract results
result = forest.predict(testDataVecs)

#Write the test results
output=pd.DataFrame(data={'id':test['id'],'sentiment':result})
output.to_csv('Word2Vec_AverageVectors.csv',index=False,quoting=3)


Fitting a random forest to labeled traning data...


In [12]:
pred=pd.read_csv('Word2Vec_AverageVectors.csv')

In [13]:
pred.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


**Need to evaluate: we found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points....**

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.

## From Words to Paragraphs, Attemp2: Clustering
Word2Vec Creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping in this way is known as **vector quantization.** We can find the centers of the word clusters using K-Means(slow) 

In [10]:
from sklearn.cluster import KMeans
import time

start=time.time()

# Set 'K' (number clusters) to be 1/5th of the vocabulary size, or an avearge of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0]/5

#Initialize a k-means object and use it to extract centroids
kmeans_clustering=KMeans(n_clusters=num_clusters)
idx=kmeans_clustering.fit_predict(word_vectors)

#Get the end time and print how long the process took
end = time.time()
elapsed = end-start
print 'Time taken for K Means Clustering:',elapsed,'seconds'

Time taken for K Means Clustering: 544.380803108 seconds


In [19]:
# Create a Word/Index dictionary, mapping each vocabulary word to a cluster number
word_centroid_map=dict(zip(model.index2word,idx))

In [32]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #Print the cluster number
    print "\nCluster %d" %cluster
    #Find all of the words for that cluster number, and print them out
    words =[]
    for i in xrange(0,len(word_centroid_map.values())):
        if (word_centroid_map.values()[i]==cluster):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'turner', u'claudette', u'colbert']

Cluster 1
[u'humiliating']

Cluster 2
[u'freeze', u'butterfly', u'soaked', u'stained', u'piercing', u'bleed']

Cluster 3
[u'extend', u'generating', u'enhance']

Cluster 4
[u'uncertain', u'unflinching', u'imperfect', u'audacious', u'uncompromising', u'proposition', u'introspective', u'undeniable', u'exploitive', u'allegorical', u'alarming', u'overload', u'unrelenting', u'organic']

Cluster 5
[u'corky']

Cluster 6
[u'1959', u'pittsburgh', u'1961', u'korda', u'1964', u'1941']

Cluster 7
[u'millionaire', u'businessman', u'menjou', u'penniless', u'billionaire', u'tycoon', u'newly', u'banker', u'socialite', u'developer', u'wealthy']

Cluster 8
[u'shelves', u'tube', u'netflix', u'ebay', u'shelf']

Cluster 9
[u'hostile', u'liberal', u'implication', u'muslim', u'conservative', u'elitist', u'general', u'islamic', u'establishment', u'serbian', u'fascist', u'pro', u'bankrupt']


In [33]:
def create_bag_of_centroids(wordlist,word_centroid_map):
    #The numberof clusters is equal to the highest cluster index in the word/centroid map
    num_centroids =max(word_centroid_map.values())+1
    #Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids=np.zeros(num_centroids,dtype='float32')
    #Loop over the words in the review. If the word is in the vocabulary, find which clusters it belongs to, and increment that clusters count by 1
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index]+=1
    #return the 'bag of centroid
    return bag_of_centroids

In [35]:
#Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids=np.zeros((train['review'].size,num_clusters),dtype='float32')
#Transform the training set reviews into bags of centroids
counter =0
for review in clean_train_reviews:
    train_centroids[counter]=create_bag_of_centroids(review,word_centroid_map)
    counter+=1
#Repeat for test reviews
counter =0
test_centroids=np.zeros((test['review'].size,num_clusters),dtype='float32')
for review in clean_test_reviews:
    test_centroids[counter]=create_bag_of_centroids(review,word_centroid_map)
    counter+=1

In [36]:
#Fit a random forest and extract predictions
forest =RandomForestClassifier(n_estimators=100)

#Fitting the forest may take a few mins
print "Fitting a random forest to labeled training data..."
forest =forest.fit(train_centroids,train['sentiment'])
result = forest.predict(test_centroids)

#Write the test results
output =pd.DataFrame(data={'id':test['id'],'sentiment':result})
output.to_csv('BagofCentroids.csv',index=False,quoting=3)

Fitting a random forest to labeled training data...


**Need to evaluate: The code above gives about the same (or slighty worse) results compared to the Bag of Words in Part1**